# Combinacion de datasets de recorridos de bicicletas

In [1]:
import pandas as pd
import numpy as np

In [2]:
data: list[pd.DataFrame] = []
for i in range(2014, 2025):
    print(f"Loading data for {i}")
    data.append(
        pd.read_csv(
            f"data/recorridos-ecobici/recorridos-realizados-{i}.csv", low_memory=False
        )
    )

print("Datasets: ", len(data))

Loading data for 2014
Loading data for 2015
Loading data for 2016
Loading data for 2017
Loading data for 2018
Loading data for 2019
Loading data for 2020
Loading data for 2021
Loading data for 2022
Loading data for 2023
Loading data for 2024
Datasets:  11


In [3]:
groups = {}
for year, df in enumerate(data, 2014):
    columns = tuple(df.columns.values.tolist())
    groups[columns] = groups.get(columns, []) + [(year, df)]

for i, group in enumerate(groups.values()):
    print("Group ", i)
    for year, df in group:
        print(year, df.shape)

Group  0
2014 (1069606, 5)
Group  1
2015 (503252, 15)
2016 (596807, 15)
2017 (1048158, 15)
Group  2
2018 (2619968, 16)
Group  3
2019 (6217004, 18)
2020 (2415597, 18)
2023 (2622331, 18)
Group  4
2021 (2860091, 19)
Group  5
2022 (2922805, 19)
Group  6
2024 (1067623, 17)


In [4]:
grouped_data: dict[tuple[int, ...], pd.DataFrame] = {}
for group in groups.values():
    as_dict = dict(group)
    grouped_data[tuple(as_dict.keys())] = pd.concat([df for df in as_dict.values()])

for group, df in grouped_data.items():
    print(group, df.shape)

(2014,) (1069606, 5)
(2015, 2016, 2017) (2148217, 15)
(2018,) (2619968, 16)
(2019, 2020, 2023) (11254932, 18)
(2021,) (2860091, 19)
(2022,) (2922805, 19)
(2024,) (1067623, 17)


In [5]:
for group, df in grouped_data.items():
    print("Group: ", group)
    for col in df.columns.values:
        print("- ", col)

Group:  (2014,)
-  ID
-  NOMBRE_ORIGEN
-  ORIGEN_FECHA
-  DESTINO_ESTACION
-  DESTINO_FECHA
Group:  (2015, 2016, 2017)
-  periodo
-  genero_usuario
-  fecha_origen_recorrido
-  id_estacion_origen
-  nombre_estacion_origen
-  long_estacion_origen
-  lat_estacion_origen
-  domicilio_estacion_origen
-  duracion_recorrido
-  fecha_destino_recorrido
-  id_estacion_destino
-  nombre_estacion_destino
-  long_estacion_destino
-  lat_estacion_destino
-  domicilio_estacion_destino
Group:  (2018,)
-  periodo
-  id_usuario
-  genero_usuario
-  fecha_origen_recorrido
-  id_estacion_origen
-  nombre_estacion_origen
-  long_estacion_origen
-  lat_estacion_origen
-  domicilio_estacion_origen
-  duracion_recorrido
-  fecha_destino_recorrido
-  id_estacion_destino
-  nombre_estacion_destino
-  long_estacion_destino
-  lat_estacion_destino
-  domicilio_estacion_destino
Group:  (2019, 2020, 2023)
-  Unnamed: 0
-  Id_recorrido
-  duracion_recorrido
-  fecha_origen_recorrido
-  id_estacion_origen
-  nombre_

In [6]:
homogeneous_groups = {}

group = (2014,)
homogeneous_groups[group] = grouped_data[group].rename(
    columns={"NOMBRE_ORIGEN": "ORIGEN", "ORIGEN_FECHA": "FECHA"}
)[["ORIGEN", "FECHA"]]

homogeneous_groups[group]["GENERO"] = np.nan

group = (2015, 2016, 2017)
homogeneous_groups[group] = grouped_data[group].rename(
    columns={
        "nombre_estacion_origen": "ORIGEN",
        "fecha_origen_recorrido": "FECHA",
        "genero_usuario": "GENERO",
    }
)[["ORIGEN", "FECHA", "GENERO"]]

group = (2018,)
homogeneous_groups[group] = grouped_data[group].rename(
    columns={
        "nombre_estacion_origen": "ORIGEN",
        "fecha_origen_recorrido": "FECHA",
        "genero_usuario": "GENERO",
    }
)[["ORIGEN", "FECHA", "GENERO"]]

group = (2019, 2020, 2023)
homogeneous_groups[group] = grouped_data[group].rename(
    columns={
        "nombre_estacion_origen": "ORIGEN",
        "fecha_origen_recorrido": "FECHA",
        "género": "GENERO",
    }
)[["ORIGEN", "FECHA", "GENERO"]]

group = (2021,)
homogeneous_groups[group] = grouped_data[group].rename(
    columns={
        "nombre_estacion_origen": "ORIGEN",
        "fecha_origen_recorrido": "FECHA",
        "género": "GENERO",
    }
)[["ORIGEN", "FECHA", "GENERO"]]

group = (2022,)
homogeneous_groups[group] = grouped_data[group].rename(
    columns={
        "nombre_estacion_origen": "ORIGEN",
        "fecha_origen_recorrido": "FECHA",
        "Género": "GENERO",
    }
)[["ORIGEN", "FECHA", "GENERO"]]

group = (2024,)
homogeneous_groups[group] = grouped_data[group].rename(
    columns={
        "nombre_estacion_origen": "ORIGEN",
        "fecha_origen_recorrido": "FECHA",
        "género": "GENERO",
    }
)[["ORIGEN", "FECHA", "GENERO"]]

generos = {
    "M": "MASCULINO",
    "F": "FEMENINO",
    "N": "OTRO",
    "MALE": "MASCULINO",
    "FEMALE": "FEMENINO",
    "OTHER": "OTRO",
}

for df in homogeneous_groups.values():
    df.replace({"GENERO": generos}, inplace=True)
    df["FECHA"] = pd.to_datetime(df["FECHA"], format="mixed")

In [7]:
homogeneous_data = pd.concat(homogeneous_groups.values())
homogeneous_data.head()

,ORIGEN,FECHA,GENERO
0,ONCE,2014-01-02 08:03:44.437,NaN
1,SUIPACHA,2014-01-02 08:33:59.873,NaN
2,CONGRESO,2014-01-02 08:43:09.263,NaN
3,VIRREY CEVALLOS,2014-01-02 08:43:37.700,NaN
4,RETIRO,2014-01-02 08:47:58.217,NaN


In [8]:
final_data = homogeneous_data.copy()
final_data["MES_NUM"] = final_data["FECHA"].dt.month
final_data["ANIO"] = final_data["FECHA"].dt.year

meses = [
    "Enero",
    "Febrero",
    "Marzo",
    "Abril",
    "Mayo",
    "Junio",
    "Julio",
    "Agosto",
    "Septiembre",
    "Octubre",
    "Noviembre",
    "Diciembre",
]
final_data["MES"] = final_data["MES_NUM"].apply(lambda x: meses[x - 1])
final_data.head()

,ORIGEN,FECHA,GENERO,MES_NUM,ANIO,MES
0,ONCE,2014-01-02 08:03:44.437,NaN,1,2014,Enero
1,SUIPACHA,2014-01-02 08:33:59.873,NaN,1,2014,Enero
2,CONGRESO,2014-01-02 08:43:09.263,NaN,1,2014,Enero
3,VIRREY CEVALLOS,2014-01-02 08:43:37.700,NaN,1,2014,Enero
4,RETIRO,2014-01-02 08:47:58.217,NaN,1,2014,Enero


In [9]:
final_data.to_csv("data/recorridos-realizados-todo.csv", index=False)
print("Archivo generado")

Archivo generado
